In [166]:
import os

import numpy as np
import pandas as pd

In [171]:
DATA_PATH = '../data/'
EXPORT_PATH = '/export/'

files = {
    'data': [
        DATA_PATH + 'baskets.parquet',
        DATA_PATH + 'coupon_index.parquet', 
        DATA_PATH + 'coupons.parquet',
        #DATA_PATH + 'prepare.parquet'
   ]
}

In [177]:
# todo: rename to Utils and break inheritance
class Helper:
    """
    Expose utility methods
    """

    def __init__(self):
        """
        Attributes:
            data: (dict) contains the loaded data sets
            mappings: (dict) todo: what kind of mappings?
        """
        self.data = {}
        self.mappings = {}

    # read files (data/exports) from disk
    # ----------------------------------------------------------------------------------
    def load(self, files: dict):
        for key, paths in files.items():
            print(key, paths)
            for path in paths:
                name_and_extension = os.path.basename(path)
                name = os.path.splitext(name_and_extension)[0]
                self[key][name] = pd.read_parquet(path)
        """
        use:
            - load files from the files-dictionary and store at correct location
        inputs:
            - files: dict
                - key: str -> location/attribute where the data should be stored
                  (e.g. data, mapping)
                - value: list or str --> filepath or list of filepaths

        return: None
        """
    # convenience functions
    # ----------------------------------------------------------------------------------
    def __getitem__(self, item):
        """
        use:
            - dict-like behaviour: self['item'] <-> self.item
            - allows to get attributes using a str
        """
        return eval(f"self.{item}")

    def __setitem__(self, item, value):
        """
        use:
            - dict-like behaviour: self['item'] = value <-> self.item = value
            - allows to set attribute values using a str
        """
        exec(f"self.{item} = value")

    def _format_time(self, seconds):
        """
        use:
            - formats seconds into str time-format: 'mm:ss'
        """
        return "{:02.0f}".format(seconds // 60) + ":" + "{:02.0f}".format(seconds % 60)

In [179]:
myHelper = Helper()
myHelper.load(files)
myHelper.data

data ['../data/baskets.parquet', '../data/coupon_index.parquet', '../data/coupons.parquet']


{'baskets':           week  shopper  product  price
 0            0        0       71    629
 1            0        0       91    605
 2            0        0      116    715
 3            0        0      123    483
 4            0        0      157    592
 ...        ...      ...      ...    ...
 68841593    89    99999      143    470
 68841594    89    99999      158    566
 68841595    89    99999      186    499
 68841596    89    99999      204    496
 68841597    89    99999      225    602
 
 [68841598 rows x 4 columns],
 'coupon_index':       week  shopper  coupon
 0       90        0       0
 2000    90        0       1
 4000    90        0       2
 6000    90        0       3
 8000    90        0       4
 ...    ...      ...     ...
 1999    90     1999       0
 3999    90     1999       1
 5999    90     1999       2
 7999    90     1999       3
 9999    90     1999       4
 
 [10000 rows x 3 columns],
 'coupons':           week  shopper  product  discount
 0            0  